# CSV example

*Last edited: 2023-12-11*

In [1]:
%%writefile data.txt
1, "Mediterranean Avenue"
2, "Baltic Avenue"
3, "Oriental Avenue"
4, "Vermont Avenue"
5, "Connecticut Avenue"
6, "St. Charles Place"

Writing data.txt


In [2]:
%%writefile csv.f90
program main
    use, intrinsic :: iso_fortran_env               ! error_unit
    implicit none

    character(len=*), parameter :: F = 'data.txt'   ! File name.
    integer,          parameter :: U = 20           ! Output unit.
    character(len=100)          :: street
    integer                     :: i, rc

    open (unit=U, action='read', file=F, iostat=rc)
    if (rc /= 0) then
        write (error_unit, '(3a, i0)') 'Reading file "', F,  &
            '" failed: error ', rc
        stop
    end if

    do
        read (U, *, iostat=rc) i, street            ! Read from file.
        if (rc /= 0) exit                           ! Exit on error.
        print '(i1, a, a)', i, ': ', trim(street)   ! Output line.
    end do

    close (U)
end program main

Writing csv.f90


In [18]:
%%bash
gfortran -O3 -s \
    -no-pie \
    -fomit-frame-pointer \
    -fno-exceptions \
    -fno-unwind-tables \
    -fno-asynchronous-unwind-tables \
    -Wl,-z,norelro \
    -Wl,--build-id=none \
    csv.f90
objcopy -R .comment -R .gnu.version -R .gnu.hash -R .note* a.out
./a.out ; echo $? ; wc -c a.out

1: Mediterranean Avenue
2: Baltic Avenue
3: Oriental Avenue
4: Vermont Avenue
5: Connecticut Avenue
6: St. Charles Place
0
10760 a.out


* -no-pie
    * Don’t produce a dynamically linked position independent executable.
* -fomit-frame-pointer
    * Don't keep the frame pointer in a register for functions that don't need one.
* -fno-exceptions
    * The -fno-exceptions flag in gfortran disables the generation of exception handling code. This means that the compiler will not produce code to handle runtime errors such as division by zero, floating-point exceptions, or invalid operations. This can result in a smaller and potentially faster executable, but it also means that your program won't be able to gracefully handle these errors.
* -fno-unwind-tables
    * The -fno-unwind-tables flag in gfortran disables the generation of unwind tables in the generated code. Unwind tables are used for stack unwinding during exception handling and are part of the exception handling mechanism. By using this flag, you can reduce the size of the executable, but it also means that the program will not have proper stack unwinding support, which can affect debugging and exception handling.
* -fno-asynchronous-unwind-tables
    * Asynchronous unwind tables are used for stack unwinding from asynchronous events, such as those triggered by a debugger or garbage collector.
* -Wl,-z,norelro
    * The -Wl,-z,norelro flag is used with the GNU linker (ld) to instruct the linker to omit the generation of the Relocation Read-Only (RELRO) sections in the executable. RELRO is a security feature that makes certain sections of the program memory read-only to prevent certain types of attacks, such as buffer overflow attack.
* -Wl,--build-id=none
    * The -Wl,--build-id=none flag is used with the GNU linker (ld) to instruct the linker to omit the generation of a unique build ID for the executable. The build ID is typically used for debugging purposes to uniquely identify different builds of the same program.

Refs.:

* <https://gcc.gnu.org/onlinedocs/gcc-4.0.3/gcc/Optimize-Options.html>
* <https://gcc.gnu.org/onlinedocs/gcc/Link-Options.html>
* <https://gcc.gnu.org/onlinedocs/gcc/Instrumentation-Options.html>

In [17]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000112:
EXEC_P, HAS_SYMS, D_PAGED
start address 0x0000000000401140

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000400040 paddr 0x0000000000400040 align 2**3
         filesz 0x00000000000002a0 memsz 0x00000000000002a0 flags r--
  INTERP off    0x00000000000002e0 vaddr 0x00000000004002e0 paddr 0x00000000004002e0 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000400000 paddr 0x0000000000400000 align 2**12
         filesz 0x0000000000000880 memsz 0x0000000000000880 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000401000 paddr 0x0000000000401000 align 2**12
         filesz 0x00000000000004d5 memsz 0x00000000000004d5 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000402000 paddr 0x0000000000402000 align 2**12
         filesz 0x00000000000000cc memsz 0x00000000000000cc flags r--
   